In [50]:
import torch
from transformers import AutoTokenizer, AutoModel
import copy

from model import Model

In [51]:
model_ckpt = "nghuyong/ernie-health-zh"

token = AutoTokenizer.from_pretrained(model_ckpt)
print(token.model_input_names)
pretrained = AutoModel.from_pretrained(model_ckpt)
print(pretrained.num_parameters())

['input_ids', 'token_type_ids', 'attention_mask']


Some weights of ErnieModel were not initialized from the model checkpoint at nghuyong/ernie-health-zh and are newly initialized: ['ernie.pooler.dense.weight', 'ernie.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


103404288


In [52]:
best_model = Model(copy.deepcopy(pretrained))
best_model.load_state_dict(torch.load('torch_model.bin'))

text = '肚子疼去什么科室？'
text_encode = token([text], return_tensors='pt')
print(text_encode)

{'input_ids': tensor([[  2, 387,  66, 124,  85,  60,  23, 174, 598,   1,   3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [53]:
# 测试通过
predict = best_model(text_encode['input_ids'],
                     text_encode['attention_mask'],
                     text_encode['token_type_ids'])
predict

tensor([[1.3717e-04, 1.3109e-04, 7.0502e-05, 9.9887e-01, 1.6484e-04, 1.1718e-04,
         1.0990e-04, 1.0285e-04, 9.1779e-05, 1.8084e-04, 2.4027e-05]],
       grad_fn=<SoftmaxBackward0>)

In [54]:
torch.max(predict)

tensor(0.9989, grad_fn=<MaxBackward1>)

In [55]:
torch.argmax(predict)

tensor(3)